In [24]:
import pandas as pd
import numpy as np



In [25]:
df_final=pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/df_final.csv")

FileNotFoundError: ignored

In [26]:
df_final.head()

NameError: ignored

In [ ]:
df_final.info()

In [ ]:
# Réduction du DataFrame à une sélection de variables CSTB + Cerema + variable cible "y" 
# + variables créées précédemment : "etaban_lot" + "qty_of_mut"
# Voir colonnne n°1 du tableau Excel : "Travail_variables"

df_selec=df_final[["idmutation",
                   "etaban_lot",
                   "qty_of_mut",
                   "y",
                   "idnatmut",
                   "datemut",
                   "anneemut",
                   "moismut",
                   "libnatmut",
                   "vefa",
                   "valeurfonc",
                   "nblot",
                   "nbcomm",
                   "l_codinsee",
                   "nblocapt",
                   "nblocdep",
                   "nbapt2pp",
                   "nbapt3pp",
                   "sbati",
                   "sbatapt",
                   "sbatact",
                   "sapt2pp",
                   "sapt3pp",
                   "scarrez",
                   "nolot",
"adedpe202006_logtype_periode_construction",
"anarnc202012_nb_log",
"anarnc202012_nb_lot_garpark",
"anarnc202012_nb_lot_tertiaire",
"anarnc202012_nb_lot_tot",
"etaban202111_label",
"mcumer202007_id",
"adedpe202006_logtype_baie_orientation",
"adedpe202006_logtype_baie_type_vitrage",
"adedpe202006_logtype_ch_gen_lib_princ",
"adedpe202006_logtype_classe_conso_ener",
"adedpe202006_logtype_classe_estim_ges",
"adedpe202006_logtype_traversant",
"adedpe202006_logtype_ecs_type_ener",
"adedpe202006_logtype_enr",
"adedpe202006_logtype_perc_surf_vitree_ext",
"adedpe202006_logtype_ph_pos_isol",
"adedpe202006_logtype_presence_balcon",
"adedpe202006_logtype_presence_climatisation",
"adedpe202006_logtype_type_batiment",
"ancqpv201410_is_qpv",
"arcthd2021t1_l_bat_type",
"cerffo2020_annee_construction",
"cerffo2020_usage_niveau_1_txt",
"cerffo2020_nb_log",
"geoarg2020_alea",
"georad2020_alea",
"igntop202103_bat_hauteur",
"insbpe2019_l_type_equipement",
"mcumer202007_dist500min",
"mcumer202007_is_mer",
"adedpe202006_logtype_s_hab"]]


In [ ]:
df_selec.head()

In [ ]:
df_selec.info()

In [ ]:
# Suppression de la variable "qty_of_mut" pq quand au lancement des modèles de classification, elle prend le pas sur toutes les autres
# et du coup aucun faux négatif, ni faux positifs ne sortent. Tous les scores = 1

df_selec=df_selec.drop("qty_of_mut", axis=1)

In [ ]:
# Variable "adedpe202006_logtype_enr" => Energies renouvelables présentes dans le bâtiment
df_selec["adedpe202006_logtype_enr"].unique()

In [ ]:
df_selec["adedpe202006_logtype_enr"].value_counts()

In [ ]:
# Seulement 36 lignes pour lesquelles la variable est renseignée. 
# Le reste est Nan  mais on peut supposer que ce ne sont pas des vraies Nan 
# Ca peut plutôt signifier l'absence d'énergies renouvelables dans le bâtiment

def filtre (x):
    if x =="solaire thermique (ecs)" :
        return 1
    elif x =="solaire photovoltaique" :
        return 1
    else : 
        return 0
    
df_selec["adedpe202006_logtype_enr"]=df_selec["adedpe202006_logtype_enr"].apply(filtre)

In [ ]:
df_selec["adedpe202006_logtype_enr"].value_counts()

In [ ]:
df_selec["cerffo2020_usage_niveau_1_txt"].value_counts()

In [ ]:
# Regroupement des dépendances et secondaires en autres sinon plantage en get dummies

df_selec["cerffo2020_usage_niveau_1_txt"]=df_selec["cerffo2020_usage_niveau_1_txt"].replace({"Dépendance":"Autres", "Secondaire":"Autres"})

In [ ]:
df_selec["cerffo2020_usage_niveau_1_txt"].value_counts()

In [ ]:
# Les variables qui concernent les dates ne vont pas passer en get_dummies. Je les supprime à ce stade 
# quitte à les réintégrer plus tard 
df_selec=df_selec.drop("datemut", axis=1)
df_selec=df_selec.drop("anneemut", axis=1)
df_selec=df_selec.drop("moismut", axis=1)

In [ ]:
df_selec["libnatmut"].unique()

In [ ]:
df_selec["vefa"].unique()

In [ ]:
#La variable "nolot" n'est plus utile. Elle a été fusionnée avec etaban dans etaban_lot dans df_final["nolot"]

df_selec=df_selec.drop("nolot", axis=1)

In [ ]:
# Je supprime aussi la variable etaban_lot qui va poser problème au moment du get_dummies. 

df_selec=df_selec.drop("etaban_lot", axis=1)

In [ ]:
df_selec["adedpe202006_logtype_periode_construction"].isna().sum()

In [ ]:
df_selec["adedpe202006_logtype_periode_construction"].unique()

In [ ]:
df_selec["cerffo2020_annee_construction"].isna().sum()

In [ ]:
#Comme année de construction possède bcp moins de na, on ne va garder que cette variable mais on va la discrétiser
df_selec["cerffo2020_annee_construction"]=pd.cut(x=df_selec["cerffo2020_annee_construction"], bins=[0,1948,1970,1989,2000,2005,2012, 2022], labels=["<1948", "1949-1970", "1970-1988", "1989-1999", "2000-2005", "2005-2012", ">2012"])
df_selec["cerffo2020_annee_construction"].unique()

In [ ]:
#suppression de la variable " adedpe202006_logtype_periode_construction "

df_selec.drop("adedpe202006_logtype_periode_construction", axis=1, inplace=True)

In [ ]:
df_selec["adedpe202006_logtype_baie_orientation"]

In [ ]:
# Simplification des valeurs de la variable "adedpe202006_logtype_baie_orientation" => 
# sa multitude de valeurs créé des différence de nombre de colonnes entre X train et X test après get dummies

df_selec["adedpe202006_logtype_baie_orientation"]=df_selec["adedpe202006_logtype_baie_orientation"].astype(str)


In [ ]:
df_selec["adedpe202006_logtype_baie_orientation"]=df_selec["adedpe202006_logtype_baie_orientation"].apply(lambda x : x.strip("()"))
df_selec["adedpe202006_logtype_baie_orientation"]

In [ ]:
df_selec["adedpe202006_logtype_baie_orientation"].unique()

In [ ]:
df_selec["adedpe202006_logtype_baie_orientation"]=df_selec["adedpe202006_logtype_baie_orientation"].apply(lambda x : x[2:])
df_selec["adedpe202006_logtype_baie_orientation"].unique()

In [ ]:
df_selec["adedpe202006_logtype_baie_orientation"]=df_selec["adedpe202006_logtype_baie_orientation"].replace({"n": "Nan"})
df_selec["adedpe202006_logtype_baie_orientation"].unique()

In [ ]:
df_selec["adedpe202006_logtype_baie_orientation"].value_counts()

In [ ]:
def filtre_sud(x) :
    if "sud" in x :
        return 1
    else : 
        return 0
    
df_selec["baie_orientation_sud"]=df_selec["adedpe202006_logtype_baie_orientation"].apply(filtre_sud)

In [ ]:
df_selec["baie_orientation_sud"].value_counts()

In [ ]:
def filtre_nord(x) :
    if "nord" in x :
        return 1
    else : 
        return 0
    
df_selec["baie_orientation_nord"]=df_selec["adedpe202006_logtype_baie_orientation"].apply(filtre_nord)

In [ ]:
df_selec["baie_orientation_nord"].value_counts()

In [ ]:
def filtre_ouest(x) :
    if "ouest" in x :
        return 1
    else : 
        return 0
    
df_selec["baie_orientation_ouest"]=df_selec["adedpe202006_logtype_baie_orientation"].apply(filtre_ouest)

In [ ]:
df_selec["baie_orientation_ouest"].value_counts()

In [ ]:
def filtre_est(x) :
    if x.startswith("est") :
        return 1
    else : 
        return 0
    
df_selec["baie_orientation_est"]=df_selec["adedpe202006_logtype_baie_orientation"].apply(filtre_est)


In [ ]:
df_selec["baie_orientation_est"].value_counts()


In [ ]:
def filtre_autre(x) :
    if "indetermine" in x:
        return 1
    elif "horizontale" in x:
        return 1
    else : 
        return 0
    
df_selec["baie_orientation_autre"]=df_selec["adedpe202006_logtype_baie_orientation"].apply(filtre_autre)


In [ ]:
df_selec["baie_orientation_autre"].value_counts()

In [ ]:
# suppression de la variable df_selec["adedpe202006_logtype_baie_orientation"]

df_selec=df_selec.drop("adedpe202006_logtype_baie_orientation", axis=1)

In [ ]:
# Simplification des valeurs de la variable "adedpe202006_logtype_ch_gen_lib_princ" => 
# sa multitude de valeurs créé des différence de nombre de colonnes entre X train et X test après get dummies

df_selec["adedpe202006_logtype_ch_gen_lib_princ"].value_counts()

In [ ]:
# Regroupement des occurences 

liste_equipement =["chaudiere bois",
                   "chaudiere energie indetermine condensation",
                   "pac eau/eau",
                   "pac geothermique",
                   "poele ou insert bois",
                   "chaudiere gpl/butane/propane standard",
                   "poele ou insert charbon",
                   "chaudiere charbon standard",
                   "poele ou insert indetermine",
                   "chauffage solaire"]

def filtre_autre_equipement (x) :
    if x in  liste_equipement :
        return "autres"
    else :
        return x
        
df_selec["adedpe202006_logtype_ch_gen_lib_princ"]=df_selec["adedpe202006_logtype_ch_gen_lib_princ"].apply(filtre_autre_equipement)
df_selec["adedpe202006_logtype_ch_gen_lib_princ"].value_counts()

In [ ]:
# Simplification des valeurs de la variable "adedpe202006_logtype_ecs_type_ener" => 
# sa multitude de valeurs créé des différences de nombre de colonnes entre X train et X test après get dummies.
df_selec["adedpe202006_logtype_ecs_type_ener"].value_counts()

In [ ]:
# Regroupement des occurences 

liste_equipement =["ecs thermodynamique electrique(pac ou ballon)",
                   "bois",
                   "ecs thermodynamique electrique(pac ou ballon) + gaz",
                   "bois + reseau de chaleur",
                   "electricite + gaz",
                   "bois + gaz",
                   "ecs thermodynamique electrique(pac ou ballon) + electricite",
                   "bois + ecs thermodynamique electrique(pac ou ballon) + gaz",
                   "autre",
                   "gpl/butane/propane",
                   "charbon + gaz",
                   "electricite + reseau de chaleur"]

def filtre_autre_equipement (x) :
    if x in  liste_equipement :
        return "autres"
    else :
        return x
        
df_selec["adedpe202006_logtype_ecs_type_ener"]=df_selec["adedpe202006_logtype_ecs_type_ener"].apply(filtre_autre_equipement)
df_selec["adedpe202006_logtype_ecs_type_ener"].value_counts()



In [ ]:
# Variable "arcthd2021t1_l_bat_type" => (arcep) Type de construction du bâtiment

df_selec["arcthd2021t1_l_bat_type"].value_counts()


In [ ]:
# On supprime cette variable pq on ne s'intéresse qu'aux bâtiments en copropriété 

df_selec=df_selec.drop("arcthd2021t1_l_bat_type", axis=1)


In [ ]:
numeric_columns=df_selec.select_dtypes(include=np.number).columns.tolist()
numeric_columns



In [ ]:
numeric_columns

In [ ]:
object_columns=df_selec.select_dtypes(include='object').columns.tolist()
object_columns


In [ ]:
for col in df_selec[object_columns]:
    print(df_selec[col].nunique())

In [ ]:
type(numeric_columns)

In [ ]:
#A quoi correspondent les colonnes qui ont 26298, 1767 et 4123 valeurs uniques  ?
print(object_columns[2],object_columns[3], object_columns[-1]) 


In [ ]:
df_selec["etaban202111_label"]

In [ ]:
#On supprime la colonne etaban202111_label" => on travaillera uniquemet avec le code etaban_lot si besoin
object_columns.remove("etaban202111_label")
df_selec=df_selec.drop("etaban202111_label", axis=1)


In [ ]:
df_selec["mcumer202007_id"]

In [ ]:
df_selec["mcumer202007_id"].nunique()

In [ ]:
df_final["mcumer202007_id"].isna().sum()

In [ ]:
df_final["mcumer202007_id"].describe()

In [ ]:
# 88130 des mutations ont un identifiant mcumer202007_id. Le reste ce sont des nan. 
# Si cet identifiant veut dire que le bâtiment est dans un périmètre ABF monument historique, autant prendre la variable mcumer202007_is_mer. 
# Si cet identifiant veut dire que le bâtiment est classé monument historique => alors pas nécessaire de le garder pq ils le sont tous dans la mesure où ils ont tous un identifiant

In [ ]:
# Décision prise de supprimer les variables "mcumer202007_id" et "mcumer202007_dist500min" pour ne garder que "mcumer202007_is_mer"

object_columns.remove("mcumer202007_id")
numeric_columns.remove("mcumer202007_dist500min")

df_selec=df_selec.drop(["mcumer202007_id","mcumer202007_dist500min"], axis=1)




In [ ]:
df_selec["insbpe2019_l_type_equipement"].nunique()

In [ ]:
df_selec["insbpe2019_l_type_equipement"]=df_selec["insbpe2019_l_type_equipement"].astype(str)
df_selec["insbpe2019_l_type_equipement"]=df_selec["insbpe2019_l_type_equipement"].apply(lambda x : x.strip("()"))
df_selec["insbpe2019_l_type_equipement"]

In [ ]:
# On ne va conserver que les occurences qui à priori occasionnent le plus de nuisances sonores pour les riverains 
# et qui par conséquent sont suceptibles de déclencher une revente

def filtre_supermarché (x):
    if "Supermarché" in x or "Épicerie" in x or "Supérette" in x or "Hypermarché" in x :
        return 1
    else :
        return 0


In [ ]:
df_selec["insbpe2019_l_type_equipement_supermarché"]=df_selec["insbpe2019_l_type_equipement"].apply(filtre_supermarché)

In [ ]:
df_selec["insbpe2019_l_type_equipement_supermarché"].value_counts()

In [ ]:
df_selec[["insbpe2019_l_type_equipement", "insbpe2019_l_type_equipement_supermarché"]]

In [ ]:
df_selec.iloc[1]["insbpe2019_l_type_equipement"]

In [ ]:
def filtre_restaurant (x):
    if "Restaurant - Restauration rapide" in x or ":Restaurant - Restauration rapide" in x or "Restaurant universitaire" in x :
        return 1
    else :
        return 0

In [ ]:
df_selec["insbpe2019_l_type_equipement_restaurant"]=df_selec["insbpe2019_l_type_equipement"].apply(filtre_restaurant)

In [ ]:
df_selec["insbpe2019_l_type_equipement_restaurant"].value_counts()

In [ ]:
def filtre_boulangerie (x):
    if "Boulangerie" in x or ":Boulangerie" in x :
        return 1
    else :
        return 0

In [ ]:
df_selec["insbpe2019_l_type_equipement_boulangerie"]=df_selec["insbpe2019_l_type_equipement"].apply(filtre_boulangerie)

In [ ]:
df_selec["insbpe2019_l_type_equipement_boulangerie"].value_counts()

In [ ]:
# Création d'une variable "insbpe2019_l_type_equipement_bruyant"
df_selec["Commerces_bruyants"]=df_selec["insbpe2019_l_type_equipement_supermarché"]+df_selec["insbpe2019_l_type_equipement_boulangerie"]+df_selec["insbpe2019_l_type_equipement_restaurant"]



In [ ]:
#On peut maintenant supprimer la colonne df_selec["insbpe2019_l_type_equipement"] 
# et les colonnes qui ont servies à créer la variable "commerces_bruyants"
to_drop=["insbpe2019_l_type_equipement", 
         "insbpe2019_l_type_equipement_supermarché",
                   "insbpe2019_l_type_equipement_restaurant",
                   "insbpe2019_l_type_equipement_boulangerie"]

df_selec=df_selec.drop(to_drop, axis=1)

In [ ]:
object_columns=df_selec.select_dtypes(include='object').columns.tolist()
object_columns

In [ ]:
for col in df_selec[object_columns]:
    print(df_selec[col].nunique())

In [ ]:
df_selec["geoarg2020_alea"].isna().sum()

In [ ]:
#La variable "geoarg2020_alea" est composée à 100% de Nan
# Aucun bâtiment en risque argile à Paris : https://assistance.brgm.fr/georisques/pourquoi-ny-til-pas-cartographie-lexposition-au-risque-retrait-gonflement-argiles-ville
#Je supprime

numeric_columns.remove("geoarg2020_alea")

df_selec=df_selec.drop("geoarg2020_alea", axis=1)

In [ ]:
df_selec.info()

In [ ]:
df_final["adedpe202006_logtype_ecs_gen_lib_princ"].value_counts()

In [ ]:
df_final["adedpe202006_logtype_ecs_type_ener"].value_counts()

In [ ]:
df_selec.to_csv(r"C:\Users\Anne\Projet Bâtiment\210622\df_selec.csv", index=False)